In [11]:
# https://www.kaggle.com/competitions/hackathon-online-data-science-mini-epidemiology/submissions

In [12]:
!pip install -q kaggle
import json

data = {"username":"tunkedsaro","key":"ff3d00dc680ff94caf6f8d4d9bcd3168"}
json_data = json.dumps(data)

# Write the JSON string to a file
file_path = "/content/kaggle.json"
with open(file_path, 'w') as json_file:
    json_file.write(json_data)

!mkdir ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/ #copy api key ---- depend on your directory -- my directory is .../colab/..
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list
!kaggle competitions download -c hackathon-online-data-science-mini-epidemiology
!unzip -q /content/hackathon-online-data-science-mini-epidemiology.zip

!pip -q install autogluon.tabular

from autogluon.tabular import TabularDataset, TabularPredictor
import time
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

ERROR: Operation cancelled by user
mkdir: cannot create directory ‘/root/.kaggle’: File exists
ref                                                    title                                        size  lastUpdated          downloadCount  voteCount  usabilityRating  
-----------------------------------------------------  ------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
waqi786/cats-dataset                                   🐾 Cats Dataset                                6KB  2024-07-31 06:12:49           1242         24  1.0              
patricklford/global-ev-sales-2010-2024                 Global EV Sales: 2010-2024                   83KB  2024-07-19 13:40:22           2087         29  1.0              
rabieelkharoua/students-performance-dataset            📚 Students Performance Dataset 📚             66KB  2024-06-12 23:09:20          26266        526  1.0              
ihelon/coffee-sales                               

In [13]:
from sklearn.metrics import mean_squared_error
import numpy as np

# Define a custom RMSE metric
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def pplot(train_df,province:str):
    # province = "Bangkok"
    Songkhla_df = train_df[train_df['PROVINCE']==province]
    plt.figure(figsize=(14,7))
    for year in Songkhla_df['Year'].unique():
        yearly_data = Songkhla_df[Songkhla_df['Year'] == year]
        plt.plot(yearly_data['Week_no'], yearly_data['Count'], label=f'Year {year}')

    plt.xlabel('Week Number')
    plt.ylabel('Count')
    plt.title(f'{province}')
    plt.legend()
    plt.grid(True)
    plt.show()

In [1]:
train_df  = pd.read_csv("/content/Train_set.csv")
test_df   = pd.read_csv("/content/Test_set.csv")
submit_df = pd.read_csv("/content/sample_submission.csv")

NameError: name 'pd' is not defined

In [ ]:
pvrecord = {i:0 for i in list(train_df['PROVINCE'].unique())}

<hr>

# Bangkok

In [ ]:
province = "Bangkok"
province_df = train_df[train_df['PROVINCE']==province]
# Split data into train and validation based on the years
years = [2014,2016,2017,2018,2019] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()

# Feature engineering: Create lag features and moving averages
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)

province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()

# Fill NaN values created by lag features
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)

# Select features and target for training
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']

# Select features and target for validation
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']

# Prepare the data for AutoGluon
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val

# Train the AutoGluon model
Bangkok = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)

# Evaluate the model on the validation data
leaderboard = Bangkok.leaderboard(val_data, extra_metrics=['rmse'])

# Get predictions on validation data
y_pred = Bangkok.predict(X_val)

# Calculate and print RMSE on validation data
rmse_value = rmse(y_val, y_pred)

pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
# Update the RMSE value for the province
pvrecord[province] = rmse_value
leaderboard

<hr>

## Samut Prakan

In [ ]:
province = "Samut Prakan"
province_df = train_df[train_df['PROVINCE']==province]
years = [2014,2015,2016,2017,2018,2019] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Samut_Prakan = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Samut_Prakan.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Samut_Prakan.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Nonthaburi

In [ ]:
province = "Nonthaburi"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2016,2017,2018,2019] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Nonthaburi = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Nonthaburi.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Nonthaburi.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

# Pathum Thani

In [ ]:
province = "Pathum Thani"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2017,2018,2019] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Pathum_Thani = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Pathum_Thani.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Pathum_Thani.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<hr>

## Phra Nakhon Si Ayutthaya

In [ ]:
province = "Phra Nakhon Si Ayutthaya"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2016,2017,2018,2019,2020] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Phra_Nakhon_Si_Ayutthaya = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Phra_Nakhon_Si_Ayutthaya.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Phra_Nakhon_Si_Ayutthaya.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<hr>

## Ang Thong

In [ ]:
province = "Ang Thong"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2016,2017,2018,2019,2020] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Ang_Thong = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Ang_Thong.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Ang_Thong.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Lopburi

In [ ]:
province = "Lopburi"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2017,2018] #* select year that is outliner ,2016
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Lopburi = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Lopburi.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Lopburi.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Sing Buri

In [ ]:
province = "Sing Buri"
province_df = train_df[train_df['PROVINCE']==province]
years = [2016,2017,2018,2019,2020] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Sing_Buri = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Sing_Buri.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Sing_Buri.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Chainat

In [ ]:
province = "Chainat"
province_df = train_df[train_df['PROVINCE']==province]
years = [2016,2017,2018,2019,2020] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Chainat = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Chainat.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Chainat.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Saraburi

In [ ]:
province = "Saraburi"
province_df = train_df[train_df['PROVINCE']==province]
years = [2014,2015,2017,2018,2019] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Saraburi = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Saraburi.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Saraburi.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Chonburi

In [ ]:
province = "Chonburi"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2016,2017,2019] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Chonburi = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Chonburi.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Chonburi.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Rayong

In [ ]:
province = "Rayong"
province_df = train_df[train_df['PROVINCE']==province]
years = [2014,2017,2018,2019] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Rayong = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Rayong.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Rayong.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Chanthaburi

In [ ]:
province = "Chanthaburi"
province_df = train_df[train_df['PROVINCE']==province]
years = [2014,2015,2016,2017,2018,2019] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Chanthaburi = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Chanthaburi.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Chanthaburi.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Trat

In [ ]:
province = "Trat"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2015,2016,2017,2018,2019,2020] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Trat = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Trat.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Trat.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Chachoengsao

In [ ]:
province = "Chachoengsao"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2015,2017,2018,2019] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Chachoengsao = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Chachoengsao.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Chachoengsao.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Prachinburi

In [ ]:
province = "Prachinburi"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2015,2016,2017,2018,2019] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Prachinburi = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Prachinburi.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Prachinburi.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Nakhon Nayok

In [ ]:
province = "Nakhon Nayok"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2016,2017,2018,2019,2020] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Nakhon_Nayok = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Nakhon_Nayok.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Nakhon_Nayok.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Sa Kaeo

In [ ]:
province = "Sa Kaeo"
province_df = train_df[train_df['PROVINCE']==province]
years = [2014,2016,2017,2018,2019] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Sa_Kaeo = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Sa_Kaeo.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Sa_Kaeo.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Nakhon Ratchasima

In [ ]:
province = "Nakhon Ratchasima"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2017,2018,2019] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Nakhon_Ratchasima = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Nakhon_Ratchasima.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Nakhon_Ratchasima.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Buriram

In [ ]:
province = "Buriram"
province_df = train_df[train_df['PROVINCE']==province]
years = [2014,2015,2016,2018,2019] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Buriram = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Buriram.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Buriram.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Surin

In [ ]:
province = "Surin"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2015,2016,2017,2018,2019] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Surin = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Surin.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Surin.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<hr>

<br>

## Sisaket

In [ ]:
province = "Sisaket"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2016,2017,2018] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Sisaket = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Sisaket.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Sisaket.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Ubon Ratchathani

In [ ]:
province = "Ubon Ratchathani"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2016,2017,2018,2019] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Ubon_Ratchathani = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Ubon_Ratchathani.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Ubon_Ratchathani.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Yasothon

In [ ]:
province = "Yasothon"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2016,2017,2018,2019,2020] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Yasothon = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Yasothon.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Yasothon.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Chaiyaphum

In [ ]:
province = "Chaiyaphum"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2015,2016,2017,2018,2019] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Chaiyaphum = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Chaiyaphum.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Chaiyaphum.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Amnat Charoen

In [ ]:
province = "Amnat Charoen"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2016,2017,2018,2019,2020] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Amnat_Charoen = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Amnat_Charoen.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Amnat_Charoen.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Bueng Kan

In [ ]:
province = "Bueng Kan"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2016,2017,2018,2019,2020] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Bueng_Kan = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Bueng_Kan.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Bueng_Kan.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Nong Bua Lamphu

In [ ]:
province = "Nong Bua Lamphu"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2016,2017,2018,2019,2020] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Nong_Bua_Lamphu = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Nong_Bua_Lamphu.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Nong_Bua_Lamphu.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

## Khon Kaen

In [ ]:
province = "Khon Kaen"
province_df = train_df[train_df['PROVINCE']==province]
years = [2014,2015,2016,2017,2018,2019] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Khon_Kaen = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Khon_Kaen.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Khon_Kaen.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Udon Thani

In [ ]:
province = "Udon Thani"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2016,2017,2018,2019,2020] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Udon_Thani = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Udon_Thani.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Udon_Thani.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Loei

In [ ]:
province = "Loei"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2016,2018,2019] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Loei = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Loei.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Loei.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Nong Khai

In [ ]:
province = "Nong Khai"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2016,2017,2018,2019,2020] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Nong_Khai = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Nong_Khai.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Nong_Khai.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Maha Sarakham

In [ ]:
province = "Maha Sarakham"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2016,2017,2018,2019,2020] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Maha_Sarakham = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Maha_Sarakham.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Maha_Sarakham.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Roi Et

In [ ]:
province = "Roi Et"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2016,2017,2018,2019] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Roi_Et = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Roi_Et.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Roi_Et.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Kalasin

In [ ]:
province = "Kalasin"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2017,2018,2019,2020] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Kalasin = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Kalasin.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Kalasin.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Sakon Nakhon

In [ ]:
province = "Sakon Nakhon"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2016,2017,2018,2019,2020] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Sakon_Nakhon = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Sakon_Nakhon.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Sakon_Nakhon.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Nakhon Phanom

In [ ]:
province = "Nakhon Phanom"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2016,2017,2018,2019,2020] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Nakhon_Phanom = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Nakhon_Phanom.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Nakhon_Phanom.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Mukdahan

In [ ]:
province = "Mukdahan"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2016,2017,2018,2019,2020] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Mukdahan = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Mukdahan.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Mukdahan.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Chiang Mai

In [ ]:
province = "Chiang Mai"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2017,2018,2020] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Chiang_Mai = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Chiang_Mai.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Chiang_Mai.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Lamphun

In [ ]:
province = "Lamphun"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2017,2018,2019,2020] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Lamphun = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Lamphun.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Lamphun.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Lampang

In [ ]:
province = "Lampang"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2016,2017,2018,2019,2020] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Lampang = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Lampang.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Lampang.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Uttaradit

In [ ]:
province = "Uttaradit"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2016,2017,2018,2019,2020] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Uttaradit = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Uttaradit.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Uttaradit.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Phrae

In [ ]:
province = "Phrae"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2017,2018,2019,2020] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Phrae = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Phrae.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Phrae.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Nan

In [ ]:
province = "Nan"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2017,2018,2020] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Nan = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Nan.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Nan.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Phayao

In [ ]:
province = "Phayao"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2017,2018,2019,2020] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Phayao = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Phayao.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Phayao.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Chiang Rai

In [ ]:
province = "Chiang Rai"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2017,2018,2019,2020] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Chiang_Rai = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Chiang_Rai.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Chiang_Rai.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Mae Hong Son

In [ ]:
province = "Mae Hong Son"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2016,2017,2018,2019,2020] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Mae_Hong_Son = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Mae_Hong_Son.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Mae_Hong_Son.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Nakhon Sawan

In [ ]:
province = "Nakhon Sawan"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2017,2018,2019,2020] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Nakhon_Sawan = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Nakhon_Sawan.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Nakhon_Sawan.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Uthai Thani

In [ ]:
province = "Uthai Thani"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2016,2017,2018,2019,2020] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Uthai_Thani = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Uthai_Thani.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Uthai_Thani.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Kamphaeng Phet

In [ ]:
province = "Kamphaeng Phet"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2017,2018,2019,2020] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Kamphaeng_Phet = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Kamphaeng_Phet.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Kamphaeng_Phet.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Tak

In [ ]:
province = "Tak"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2016,2017,2018,2019,2020] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Tak = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Tak.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Tak.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Sukhothai

In [ ]:
province = "Sukhothai"
province_df = train_df[train_df['PROVINCE']==province]
years = [2014,2015,2016,2017,2018,2019] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Sukhothai = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Sukhothai.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Sukhothai.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Phitsanulok

In [ ]:
province = "Phitsanulok"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2017,2018,2019] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Phitsanulok = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Phitsanulok.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Phitsanulok.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Phichit

In [ ]:
province = "Phichit"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2017,2018,2019,2020] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Phichit = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Phichit.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Phichit.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Phetchabun

In [ ]:
province = "Phetchabun"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2016,2017,2018,2019,2020] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Phetchabun = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Phetchabun.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Phetchabun.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Ratchaburi

In [ ]:
province = "Ratchaburi"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2016,2017,2018,2019,2020] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Ratchaburi = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Ratchaburi.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Ratchaburi.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Kanchanaburi

In [ ]:
province = "Kanchanaburi"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2016,2017,2018,2019,2020] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Kanchanaburi = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Kanchanaburi.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Kanchanaburi.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Suphan Buri

In [ ]:
province = "Suphan Buri"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2016,2017,2018,2019,2020] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Suphan_Buri = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Suphan_Buri.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Suphan_Buri.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Nakhon Pathom

In [ ]:
province = "Nakhon Pathom"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2016,2017,2018,2019] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Nakhon_Pathom = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Nakhon_Pathom.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Nakhon_Pathom.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Samut Sakhon

In [ ]:
province = "Samut Sakhon"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2016,2017,2018,2019,2020] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Samut_Sakhon = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Samut_Sakhon.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Samut_Sakhon.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Samut Songkhram

In [ ]:
province = "Samut Songkhram"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2016,2017,2018,2019,2020] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Samut_Songkhram = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Samut_Songkhram.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Samut_Songkhram.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Phetchaburi

In [ ]:
province = "Phetchaburi"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2016,2017,2018,2019,2020] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Phetchaburi = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Phetchaburi.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Phetchaburi.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Prachuap Khiri Khan

In [ ]:
province = "Prachuap Khiri Khan"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2015,2016,2017,2018,2019,2020] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Prachuap_Khiri_Khan = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Prachuap_Khiri_Khan.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Prachuap_Khiri_Khan.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Nakhon Si Thammarat

In [ ]:
province = "Nakhon Si Thammarat"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2016,2017,2018,2019] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Nakhon_Si_Thammarat = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Nakhon_Si_Thammarat.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Nakhon_Si_Thammarat.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Krabi

In [ ]:
province = "Krabi"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2016,2017,2018,2019] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Krabi = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Krabi.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Krabi.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Phang Nga

In [ ]:
province = "Phang Nga"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2016,2017,2018,2019,2020] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Phang_Nga = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Phang_Nga.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Phang_Nga.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Phuket

In [ ]:
province = "Phuket"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2016,2017,2018,2019,2020] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Phuket = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Phuket.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Phuket.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Surat Thani

In [ ]:
province = "Surat Thani"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2016,2018,2019] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Surat_Thani = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Surat_Thani.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Surat_Thani.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Ranong

In [ ]:
province = "Ranong"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2016,2017,2018,2019,2020] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Ranong = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Ranong.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Ranong.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Chumphon

In [ ]:
province = "Chumphon"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2016,2017,2018,2019] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Chumphon = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Chumphon.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Chumphon.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Songkhla

In [ ]:
province = "Songkhla"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2016,2017,2018,2019] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Songkhla = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Songkhla.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Songkhla.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Satun

In [ ]:
province = "Satun"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2016,2017,2018,2019,2020] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Satun = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Satun.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Satun.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Trang

In [ ]:
province = "Trang"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2016,2017,2018,2019,2020] #* select year that is outliner
[years[-1]]    = [2020]
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Trang = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Trang.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Trang.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Phatthalung

In [ ]:
province = "Phatthalung"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2016,2017,2018,2019] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Phatthalung = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Phatthalung.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Phatthalung.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Pattani

In [ ]:
province = "Pattani"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2016,2017,2018,2019,2020] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Pattani = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Pattani.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Pattani.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Yala


In [ ]:
province = "Yala"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2016,2017,2018,2019] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Yala = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Yala.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Yala.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<br>

## Narathiwat

In [ ]:
province = "Narathiwat"
province_df = train_df[train_df['PROVINCE']==province]
years = [2013,2014,2015,2016,2017,2018,2019] #* select year that is outliner
province_train_df = province_df[province_df['Year'].isin(years[:-1])].copy()
province_val_df = province_df[province_df['Year'].isin([years[-1]])].copy()
for lag in range(1, 4):
    province_train_df.loc[:, f'Lag_{lag}'] = province_train_df['Count'].shift(lag)
    province_val_df.loc[:, f'Lag_{lag}'] = province_val_df['Count'].shift(lag)
province_train_df.loc[:, 'Moving_Avg_3'] = province_train_df['Count'].rolling(window=3).mean()
province_val_df.loc[:, 'Moving_Avg_3'] = province_val_df['Count'].rolling(window=3).mean()
province_train_df.fillna(0, inplace=True)
province_val_df.fillna(0, inplace=True)
X_train = province_train_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_train = province_train_df['Count']
X_val = province_val_df[['Year', 'PROVINCE', 'Week_no', 'Lag_1', 'Lag_2', 'Lag_3', 'Moving_Avg_3']]
y_val = province_val_df['Count']
train_data = X_train.copy()
train_data['Count'] = y_train
val_data = X_val.copy()
val_data['Count'] = y_val
Narathiwat = TabularPredictor(label='Count', problem_type='regression', eval_metric='rmse').fit(train_data)
leaderboard = Narathiwat.leaderboard(val_data, extra_metrics=['rmse'])
y_pred = Narathiwat.predict(X_val)
rmse_value = rmse(y_val, y_pred)
pplot(pd.concat([province_train_df,province_val_df]),province)
print(f"\nRMSE: {rmse_value}")
pvrecord[province] = rmse_value
leaderboard

<hr><hr>

# Predict switching

In [ ]:
# df = pd.read_csv("/content/sample_submission.csv")
# for idx,row in df[3:].iterrows():
#     data = test_df[test_df['_id']==row['_id']].drop(columns=['_id','Disease'])
#     # print(data.PROVINCE)
#     province = data.PROVINCE.iloc[0]
#     # print(province)
#     if province == "Bangkok":
#         df.at[idx,'Pred'] = Bangkok.predict(data)
#     elif province == "Samut Prakan":
#         df.at[idx,'Pred'] = Samut_Prakan.predict(data)
#     elif province == "Nonthaburi":
#         df.at[idx,'Pred'] = Nonthaburi.predict(data)
#     elif province == "Pathum Thani":
#         df.at[idx,'Pred'] = Pathum_Thani.predict(data)
#     elif province == "Phra Nakhon Si Ayutthaya":
#         df.at[idx,'Pred'] = Phra_Nakhon_Si_Ayutthaya.predict(data)
#     elif province == "Ang Thong":
#         df.at[idx,'Pred'] = Ang_Thong.predict(data)
#     elif province == "Lopburi":
#         df.at[idx,'Pred'] = Lopburi.predict(data)
#     elif province == "Sing Buri":
#         df.at[idx,'Pred'] = Sing_Buri.predict(data)
#     elif province == "Chainat":
#         df.at[idx,'Pred'] = Chainat.predict(data)
#     elif province == "Saraburi":
#         df.at[idx,'Pred'] = Saraburi.predict(data)
#     elif province == "Chonburi":
#         df.at[idx,'Pred'] = Chonburi.predict(data)
#     elif province == "Rayong":
#         df.at[idx,'Pred'] = Rayong.predict(data)
#     elif province == "Chanthaburi":
#         df.at[idx,'Pred'] = Chanthaburi.predict(data)
#     elif province == "Trat":
#         df.at[idx,'Pred'] = Trat.predict(data)
#     elif province == "Chachoengsao":
#         df.at[idx,'Pred'] = Chachoengsao.predict(data)
#     elif province == "Prachinburi":
#         df.at[idx,'Pred'] = Prachinburi.predict(data)
#     elif province == "Nakhon Nayok":
#         df.at[idx,'Pred'] = Nakhon_Nayok.predict(data)
#     elif province == "Sa Kaeo":
#         df.at[idx,'Pred'] = Sa_Kaeo.predict(data)
#     elif province == "Nakhon Ratchasima":
#         df.at[idx,'Pred'] = Nakhon_Ratchasima.predict(data)
#     elif province == "Buriram":
#         df.at[idx,'Pred'] = Buriram.predict(data)
#     elif province == "Surin":
#         df.at[idx,'Pred'] = Surin.predict(data)
#     elif province == "Sisaket":
#         df.at[idx,'Pred'] = Sisaket.predict(data)
#     elif province == "Ubon Ratchathani":
#         df.at[idx,'Pred'] = Ubon_Ratchathani.predict(data)
#     elif province == "Yasothon":
#         df.at[idx,'Pred'] = Yasothon.predict(data)
#     elif province == "Chaiyaphum":
#         df.at[idx,'Pred'] = Chaiyaphum.predict(data)
#     elif province == "Amnat Charoen":
#         df.at[idx,'Pred'] = Amnat_Charoen.predict(data)
#     elif province == "Bueng Kan":
#         df.at[idx,'Pred'] = Bueng_Kan.predict(data)
#     elif province == "Nong Bua Lamphu":
#         df.at[idx,'Pred'] = Nong_Bua_Lamphu.predict(data)
#     elif province == "Khon Kaen":
#         df.at[idx,'Pred'] = Khon_Kaen.predict(data)
#     elif province == "Udon Thani":
#         df.at[idx,'Pred'] = Udon_Thani.predict(data)
#     elif province == "Loei":
#         df.at[idx,'Pred'] = Loei.predict(data)
#     elif province == "Nong Khai":
#         df.at[idx,'Pred'] = Nong_Khai.predict(data)
#     elif province == "Maha Sarakham":
#         df.at[idx,'Pred'] = Maha_Sarakham.predict(data)
#     elif province == "Roi Et":
#         df.at[idx,'Pred'] = Roi_Et.predict(data)
#     elif province == "Kalasin":
#         df.at[idx,'Pred'] = Kalasin.predict(data)
#     elif province == "Sakon Nakhon":
#         df.at[idx,'Pred'] = Sakon_Nakhon.predict(data)
#     elif province == "Nakhon Phanom":
#         df.at[idx,'Pred'] = Nakhon_Phanom.predict(data)
#     elif province == "Mukdahan":
#         df.at[idx,'Pred'] = Mukdahan.predict(data)
#     elif province == "Chiang Mai":
#         df.at[idx,'Pred'] = Chiang_Mai.predict(data)
#     elif province == "Lamphun":
#         df.at[idx,'Pred'] = Lamphun.predict(data)
#     elif province == "Lampang":
#         df.at[idx,'Pred'] = Lampang.predict(data)
#     elif province == "Uttaradit":
#         df.at[idx,'Pred'] = Uttaradit.predict(data)
#     elif province == "Phrae":
#         df.at[idx,'Pred'] = Phrae.predict(data)
#     elif province == "Nan":
#         df.at[idx,'Pred'] = Nan.predict(data)
#     elif province == "Phayao":
#         df.at[idx,'Pred'] = Phayao.predict(data)
#     elif province == "Chiang Rai":
#         df.at[idx,'Pred'] = Chiang_Rai.predict(data)
#     elif province == "Mae Hong Son":
#         df.at[idx,'Pred'] = Mae_Hong_Son.predict(data)
#     elif province == "Nakhon Sawan":
#         df.at[idx,'Pred'] = Nakhon_Sawan.predict(data)
#     elif province == "Uthai Thani":
#         df.at[idx,'Pred'] = Uthai_Thani.predict(data)
#     elif province == "Kamphaeng Phet":
#         df.at[idx,'Pred'] = Kamphaeng_Phet.predict(data)
#     elif province == "Tak":
#         df.at[idx,'Pred'] = Tak.predict(data)
#     elif province == "Sukhothai":
#         df.at[idx,'Pred'] = Sukhothai.predict(data)
#     elif province == "Phitsanulok":
#         df.at[idx,'Pred'] = Phitsanulok.predict(data)
#     elif province == "Phichit":
#         df.at[idx,'Pred'] = Phichit.predict(data)
#     elif province == "Phetchabun":
#         df.at[idx,'Pred'] = Phetchabun.predict(data)
#     elif province == "Ratchaburi":
#         df.at[idx,'Pred'] = Ratchaburi.predict(data)
#     elif province == "Kanchanaburi":
#         df.at[idx,'Pred'] = Kanchanaburi.predict(data)
#     elif province == "Suphan Buri":
#         df.at[idx,'Pred'] = Suphan_Buri.predict(data)
#     elif province == "Nakhon Pathom":
#         df.at[idx,'Pred'] = Nakhon_Pathom.predict(data)
#     elif province == "Samut Sakhon":
#         df.at[idx,'Pred'] = Samut_Sakhon.predict(data)
#     elif province == "Samut Songkhram":
#         df.at[idx,'Pred'] = Samut_Songkhram.predict(data)
#     elif province == "Phetchaburi":
#         df.at[idx,'Pred'] = Phetchaburi.predict(data)
#     elif province == "Prachuap Khiri Khan":
#         df.at[idx,'Pred'] = Prachuap_Khiri_Khan.predict(data)
#     elif province == "Nakhon Si Thammarat":
#         df.at[idx,'Pred'] = Nakhon_Si_Thammarat.predict(data)
#     elif province == "Krabi":
#         df.at[idx,'Pred'] = Krabi.predict(data)
#     elif province == "Phang Nga":
#         df.at[idx,'Pred'] = Phang_Nga.predict(data)
#     elif province == "Phuket":
#         df.at[idx,'Pred'] = Phuket.predict(data)
#     elif province == "Surat Thani":
#         df.at[idx,'Pred'] = Surat_Thani.predict(data)
#     elif province == "Ranong":
#         df.at[idx,'Pred'] = Ranong.predict(data)
#     elif province == "Chumphon":
#         df.at[idx,'Pred'] = Chumphon.predict(data)
#     elif province == "Songkhla":
#         df.at[idx,'Pred'] = Songkhla.predict(data)
#     elif province == "Satun":
#         df.at[idx,'Pred'] = Satun.predict(data)
#     elif province == "Trang":
#         df.at[idx,'Pred'] = Trang.predict(data)
#     elif province == "Phatthalung":
#         df.at[idx,'Pred'] = Phatthalung.predict(data)
#     elif province == "Pattani":
#         df.at[idx,'Pred'] = Pattani.predict(data)
#     elif province == "Yala":
#         df.at[idx,'Pred'] = Yala.predict(data)
#     elif province == "Narathiwat":
#         df.at[idx,'Pred'] = Narathiwat.predict(data)
#     else:
#         print(f"\n\n\nData issue detected in row {idx}. Interrupting process.")
#         break
#         # df.at[idx,'Pred'] = "???"

In [ ]:
df = pd.read_csv("/content/sample_submission.csv")
# Add new columns using .loc
df.loc[:, 'Lag_1'] = 0.0
df.loc[:, 'Lag_2'] = 0.0
df.loc[:, 'Lag_3'] = 0.0
df.loc[:, 'Moving_Avg_3'] = 0.0
df

In [ ]:
# df = pd.read_csv("/content/sample_submission.csv")
for idx,row in df[3:].iterrows():
    data = test_df[test_df['_id']==row['_id']].drop(columns=['_id','Disease'])
    # print(data.PROVINCE)
    province = data.PROVINCE.iloc[0]
    # print(province)
    if province == "Bangkok":
        result = Bangkok.predict(data)
    elif province == "Samut Prakan":
        result = Samut_Prakan.predict(data)
    elif province == "Nonthaburi":
        result = Nonthaburi.predict(data)
    elif province == "Pathum Thani":
        result = Pathum_Thani.predict(data)
    elif province == "Phra Nakhon Si Ayutthaya":
        result = Phra_Nakhon_Si_Ayutthaya.predict(data)
    elif province == "Ang Thong":
        result = Ang_Thong.predict(data)
    elif province == "Lopburi":
        result = Lopburi.predict(data)
    elif province == "Sing Buri":
        result = Sing_Buri.predict(data)
    elif province == "Chainat":
        result = Chainat.predict(data)
    elif province == "Saraburi":
        result = Saraburi.predict(data)
    elif province == "Chonburi":
        result = Chonburi.predict(data)
    elif province == "Rayong":
        result = Rayong.predict(data)
    elif province == "Chanthaburi":
        result = Chanthaburi.predict(data)
    elif province == "Trat":
        result = Trat.predict(data)
    elif province == "Chachoengsao":
        result = Chachoengsao.predict(data)
    elif province == "Prachinburi":
        result = Prachinburi.predict(data)
    elif province == "Nakhon Nayok":
        result = Nakhon_Nayok.predict(data)
    elif province == "Sa Kaeo":
        result = Sa_Kaeo.predict(data)
    elif province == "Nakhon Ratchasima":
        result = Nakhon_Ratchasima.predict(data)
    elif province == "Buriram":
        result = Buriram.predict(data)
    elif province == "Surin":
        result = Surin.predict(data)
    elif province == "Sisaket":
        result = Sisaket.predict(data)
    elif province == "Ubon Ratchathani":
        result = Ubon_Ratchathani.predict(data)
    elif province == "Yasothon":
        result = Yasothon.predict(data)
    elif province == "Chaiyaphum":
        result = Chaiyaphum.predict(data)
    elif province == "Amnat Charoen":
        result = Amnat_Charoen.predict(data)
    elif province == "Bueng Kan":
        result = Bueng_Kan.predict(data)
    elif province == "Nong Bua Lamphu":
        result = Nong_Bua_Lamphu.predict(data)
    elif province == "Khon Kaen":
        result = Khon_Kaen.predict(data)
    elif province == "Udon Thani":
        result = Udon_Thani.predict(data)
    elif province == "Loei":
        result = Loei.predict(data)
    elif province == "Nong Khai":
        result = Nong_Khai.predict(data)
    elif province == "Maha Sarakham":
        result = Maha_Sarakham.predict(data)
    elif province == "Roi Et":
        result = Roi_Et.predict(data)
    elif province == "Kalasin":
        result = Kalasin.predict(data)
    elif province == "Sakon Nakhon":
        result = Sakon_Nakhon.predict(data)
    elif province == "Nakhon Phanom":
        result = Nakhon_Phanom.predict(data)
    elif province == "Mukdahan":
        result = Mukdahan.predict(data)
    elif province == "Chiang Mai":
        result = Chiang_Mai.predict(data)
    elif province == "Lamphun":
        result = Lamphun.predict(data)
    elif province == "Lampang":
        result = Lampang.predict(data)
    elif province == "Uttaradit":
        result = Uttaradit.predict(data)
    elif province == "Phrae":
        result = Phrae.predict(data)
    elif province == "Nan":
        result = Nan.predict(data)
    elif province == "Phayao":
        result = Phayao.predict(data)
    elif province == "Chiang Rai":
        result = Chiang_Rai.predict(data)
    elif province == "Mae Hong Son":
        result = Mae_Hong_Son.predict(data)
    elif province == "Nakhon Sawan":
        result = Nakhon_Sawan.predict(data)
    elif province == "Uthai Thani":
        result = Uthai_Thani.predict(data)
    elif province == "Kamphaeng Phet":
        result = Kamphaeng_Phet.predict(data)
    elif province == "Tak":
        result = Tak.predict(data)
    elif province == "Sukhothai":
        result = Sukhothai.predict(data)
    elif province == "Phitsanulok":
        result = Phitsanulok.predict(data)
    elif province == "Phichit":
        result = Phichit.predict(data)
    elif province == "Phetchabun":
        result = Phetchabun.predict(data)
    elif province == "Ratchaburi":
        result = Ratchaburi.predict(data)
    elif province == "Kanchanaburi":
        result = Kanchanaburi.predict(data)
    elif province == "Suphan Buri":
        result = Suphan_Buri.predict(data)
    elif province == "Nakhon Pathom":
        result = Nakhon_Pathom.predict(data)
    elif province == "Samut Sakhon":
        result = Samut_Sakhon.predict(data)
    elif province == "Samut Songkhram":
        result = Samut_Songkhram.predict(data)
    elif province == "Phetchaburi":
        result = Phetchaburi.predict(data)
    elif province == "Prachuap Khiri Khan":
        result = Prachuap_Khiri_Khan.predict(data)
    elif province == "Nakhon Si Thammarat":
        result = Nakhon_Si_Thammarat.predict(data)
    elif province == "Krabi":
        result = Krabi.predict(data)
    elif province == "Phang Nga":
        result = Phang_Nga.predict(data)
    elif province == "Phuket":
        result = Phuket.predict(data)
    elif province == "Surat Thani":
        result = Surat_Thani.predict(data)
    elif province == "Ranong":
        result = Ranong.predict(data)
    elif province == "Chumphon":
        result = Chumphon.predict(data)
    elif province == "Songkhla":
        result = Songkhla.predict(data)
    elif province == "Satun":
        result = Satun.predict(data)
    elif province == "Trang":
        result = Trang.predict(data)
    elif province == "Phatthalung":
        result = Phatthalung.predict(data)
    elif province == "Pattani":
        result = Pattani.predict(data)
    elif province == "Yala":
        result = Yala.predict(data)
    elif province == "Narathiwat":
        result = Narathiwat.predict(data)
    else:
        print(f"\n\n\nData issue detected in row {idx}. Interrupting process.")
        break

    # Ensure result is a scalar value
    if isinstance(result, pd.Series):
        result = result.iloc[0]

    # Ensure result is not negative
    if result < 1:
        result = 1.0

    # Assign the result to the dataframe
    df.at[idx, 'Pred'] = result

In [ ]:
df

In [ ]:
version = '2.1.3'
formatted_time = time.strftime("%H.%M", time.localtime())
formatted_time = str(int(formatted_time.split(".")[0])+7)+formatted_time.split(".")[1]
formatted_time
csv_name = f'v.{version}.{formatted_time}.csv'
print(csv_name)
df.to_csv(csv_name,index=False)

In [ ]:
pvrecord

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
total_rmse = sum(pvrecord.values())
print(f"Total RMSE: {total_rmse}")

# Creating a bar plot
fig, ax = plt.subplots(figsize=(20, 6))
ax.bar(pvrecord.keys(), pvrecord.values(), color='skyblue')
ax.set_xlabel('Province')
ax.set_ylabel('RMSE')
ax.set_title('RMSE Values by Province')
ax.set_ylim(0, max(pvrecord.values()) + 10)

# Rotate the x-axis labels
plt.xticks(rotation=90)

# Adding the RMSE values on top of the bars
for i, v in enumerate(pvrecord.values()):
    ax.text(i, v + 1, f'{v:.2f}', ha='center')

# Display the plot
plt.grid(alpha=0.3)
plt.show()


In [ ]:
pvrecord0  = {'Bangkok': 76.36063676449949,
 'Samut Prakan': 13.049635977070057,
 'Nonthaburi': 7.249497713985841,
 'Pathum Thani': 8.574577749408144,
 'Phra Nakhon Si Ayutthaya': 8.763172056558089,
 'Ang Thong': 3.5622489642936315,
 'Lopburi': 7.394539554350971,
 'Sing Buri': 6.253745392561713,
 'Chainat': 4.850372128317462,
 'Saraburi': 9.004934429518402,
 'Chonburi': 10.784751151284173,
 'Rayong': 29.29362612877891,
 'Chanthaburi': 9.039207586402009,
 'Trat': 8.967513750095625,
 'Chachoengsao': 10.776458397495487,
 'Prachinburi': 10.575305865941772,
 'Nakhon Nayok': 3.652860249914554,
 'Sa Kaeo': 8.265378298643776,
 'Nakhon Ratchasima': 24.678096716895894,
 'Buriram': 10.444504297959122,
 'Surin': 24.616549364792785,
 'Sisaket': 12.624889186982282,
 'Ubon Ratchathani': 18.02407231485729,
 'Yasothon': 5.259409681477039,
 'Chaiyaphum': 19.807608785001023,
 'Amnat Charoen': 5.878235959045015,
 'Bueng Kan': 3.7951607901835005,
 'Nong Bua Lamphu': 3.327853211591088,
 'Khon Kaen': 18.12098161888682,
 'Udon Thani': 6.412187338357021,
 'Loei': 5.8572859313625605,
 'Nong Khai': 4.296222163775307,
 'Maha Sarakham': 7.971467665882491,
 'Roi Et': 9.630361738476637,
 'Kalasin': 6.6386293797418015,
 'Sakon Nakhon': 5.16666162320079,
 'Nakhon Phanom': 4.059842034208891,
 'Mukdahan': 4.6688712445788125,
 'Chiang Mai': 15.419972781224041,
 'Lamphun': 7.53656421144407,
 'Lampang': 7.957290790165344,
 'Uttaradit': 3.397264339792317,
 'Phrae': 4.495905692917569,
 'Nan': 9.740079675824619,
 'Phayao': 6.372662040035755,
 'Chiang Rai': 11.936946678828246,
 'Mae Hong Son': 5.371809629955286,
 'Nakhon Sawan': 11.906167302534225,
 'Uthai Thani': 3.9118348052198804,
 'Kamphaeng Phet': 9.525922515895905,
 'Tak': 5.464342176058535,
 'Sukhothai': 9.421698008380176,
 'Phitsanulok': 9.6757096172131,
 'Phichit': 8.118654864730004,
 'Phetchabun': 8.282681051272142,
 'Ratchaburi': 5.9871252340905965,
 'Kanchanaburi': 6.699083833956905,
 'Suphan Buri': 4.831099975430087,
 'Nakhon Pathom': 9.182769187059774,
 'Samut Sakhon': 4.596189361292357,
 'Samut Songkhram': 3.317848658454171,
 'Phetchaburi': 4.91742565067919,
 'Prachuap Khiri Khan': 9.82226000037338,
 'Nakhon Si Thammarat': 14.892790755064764,
 'Krabi': 5.511365974780709,
 'Phang Nga': 3.48083277749657,
 'Phuket': 5.265380505321653,
 'Surat Thani': 15.433859756576167,
 'Ranong': 3.5370765485472693,
 'Chumphon': 7.3176919374148035,
 'Songkhla': 11.556063031083653,
 'Satun': 4.058941239721519,
 'Trang': 6.62534554042965,
 'Phatthalung': 6.907408901052155,
 'Pattani': 4.455146604571581,
 'Yala': 5.440995573277607,
 'Narathiwat': 4.089186511786015}

pvrecord1  = {'Bangkok': 39.23794010974002,
 'Samut Prakan': 10.166167116762558,
 'Nonthaburi': 6.438546710645192,
 'Pathum Thani': 7.223013573266722,
 'Phra Nakhon Si Ayutthaya': 8.790417107404064,
 'Ang Thong': 3.588964622360581,
 'Lopburi': 7.7765270352184315,
 'Sing Buri': 3.871741006042545,
 'Chainat': 3.814785125760958,
 'Saraburi': 7.342297229416183,
 'Chonburi': 7.737617588243486,
 'Rayong': 13.218722445420894,
 'Chanthaburi': 6.719756876386463,
 'Trat': 3.7734066171864917,
 'Chachoengsao': 4.652566579209719,
 'Prachinburi': 6.6056278793465,
 'Nakhon Nayok': 3.6528602519031965,
 'Sa Kaeo': 5.575370759440385,
 'Nakhon Ratchasima': 18.058206829415482,
 'Buriram': 6.09719570249447,
 'Surin': 9.651037522096336,
 'Sisaket': 7.1902850356626775,
 'Ubon Ratchathani': 10.9947399822635,
 'Yasothon': 5.259409681477039,
 'Chaiyaphum': 6.986343350571975,
 'Amnat Charoen': 5.965155752019958,
 'Bueng Kan': 3.795099026970346,
 'Nong Bua Lamphu': 3.327853211591088,
 'Khon Kaen': 9.085570610846142,
 'Udon Thani': 5.506419662001304,
 'Loei': 5.68791236110198,
 'Nong Khai': 4.221642355429842,
 'Maha Sarakham': 7.971294758469244,
 'Roi Et': 8.705422231574858,
 'Kalasin': 5.883256080259316,
 'Sakon Nakhon': 5.173248790364447,
 'Nakhon Phanom': 4.029618858229757,
 'Mukdahan': 4.433071167366718,
 'Chiang Mai': 11.168174967712858,
 'Lamphun': 6.613175825838823,
 'Lampang': 7.8257110785908806,
 'Uttaradit': 3.407559889807749,
 'Phrae': 4.554907928774935,
 'Nan': 6.6971402509959965,
 'Phayao': 6.503879832812334,
 'Chiang Rai': 8.292998313562533,
 'Mae Hong Son': 5.356883066846493,
 'Nakhon Sawan': 8.71397352735358,
 'Uthai Thani': 3.970558824132814,
 'Kamphaeng Phet': 7.294665567627883,
 'Tak': 5.464342427832331,
 'Sukhothai': 6.380434604275281,
 'Phitsanulok': 8.204286529281493,
 'Phichit': 8.185863175596678,
 'Phetchabun': 8.367170420732949,
 'Ratchaburi': 5.9871252340905965,
 'Kanchanaburi': 6.67740614140352,
 'Suphan Buri': 4.794184458062974,
 'Nakhon Pathom': 8.896198958690979,
 'Samut Sakhon': 4.596189361292357,
 'Samut Songkhram': 3.3178486706368284,
 'Phetchaburi': 4.91742565067919,
 'Prachuap Khiri Khan': 6.219526766900077,
 'Nakhon Si Thammarat': 9.219175949927125,
 'Krabi': 3.630385827362823,
 'Phang Nga': 3.48083277749657,
 'Phuket': 5.265380505321653,
 'Surat Thani': 6.137177691147416,
 'Ranong': 3.5370765485472693,
 'Chumphon': 6.112417766452195,
 'Songkhla': 8.192380983732399,
 'Satun': 4.058941239721519,
 'Trang': 6.62534554042965,
 'Phatthalung': 5.027619864847653,
 'Pattani': 4.470105758413418,
 'Yala': 4.404534202722687,
 'Narathiwat': 3.6196155450344247}

 # Create a DataFrame from the dictionaries
comparison_df = pd.DataFrame([pvrecord0, pvrecord1], index=['pvrecord0', 'pvrecord1']).T
comparison_df['Difference'] = comparison_df['pvrecord0'] - comparison_df['pvrecord1']

# Display the DataFrame
print(comparison_df)

# Create a bar plot to visualize the comparison
comparison_df.plot(kind='bar', figsize=(20, 10))
plt.title('Comparison of RMSE Values')
plt.xlabel('Province')
plt.ylabel('RMSE')
plt.xticks(rotation=90)
plt.legend(title='Record')
plt.tight_layout()
plt.grid(alpha=0.3)
# Show the plot
plt.show()